Reads logs created by the secondary task app, extracts primary task info and secondary task info, and puts the data in the Postgresql database. 

NOTE: before running this script you should ensure that the database existis and that it contains tables primary_task and secondary_task. File table_create.sql contains commands for creating the necessary tables.

In [8]:
import psycopg2
import sys
import re
import os
from datetime import datetime

In [9]:
# Reset the whole process - taskIds start from zero
def reset_global():
    global SEED_TASK_ID
    global SEED_SEC_TASK_ID
    SEED_TASK_ID = 224
    SEED_SEC_TASK_ID = 1730

def reset_user():
    global client_id
    client_id = ""

# For each new Task - reset all task-related params
def reset_task():
    global start_time
    global end_time
    global state_prim
    global state_sec
    global task_type
    global label
    global finished
    global num_all_correct
    global num_correct
    global num_incorrect
    global curr_task_id
    global curr_sec_task_id
    global task_load_index

    start_time = 0
    end_time = 0
    num_correct = 0
    num_incorrect = 0
    num_all_correct = 0
    label = ""
    task_type = ""
    task_load_index = 0
    finished = True
    state_prim = 0
    state_sec = 0
    curr_task_id = -1
    curr_sec_task_id = -1


In [10]:
# CHANGE YOUR DB PARAMETERS HERE
dsn_database = "veljkodb"
dsn_hostname = "127.0.0.1"
dsn_port = "5432"
dsn_uid = "veljkopg"     
dsn_pwd = "pntdcn15"

In [11]:
def connect_to_db():
    global cursor
    global conn
    try:
        conn_string = "host="+dsn_hostname+" port="+dsn_port+" dbname="+dsn_database+" user="+dsn_uid+" password="+dsn_pwd
        print "Connecting to database\n  ->%s" % (conn_string)
        conn=psycopg2.connect(conn_string)
        cursor = conn.cursor()
        print "Connected!\n"
    except:
        print "Unable to connect to the database."
    return cursor

In [12]:
# Each line of text in the file should be processed, so that primary and secondary tasks are identified

def parse(line):
    global curr_task_id
    global curr_sec_task_id
    global num_correct 
    global num_incorrect
    global label
    global finished
    global task_load_index
    global start_time
    global start_time_sec
    global end_time
    global state_prim
    global state_sec
    global task_type
    global cursor
     
    mHP = re.compile(r"(\S+), Hidden Pattern Question Slide, HiddenPattern(\d).txt, (\w+)," )
    mHPres = mHP.search(line)
    if mHPres and state_prim == 0:
        task_type = 'HP'
        label = mHPres.group(3).lower()
        start_time = convert_to_epoch(mHPres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id 
 
    mFA = re.compile(r"(\S+), Finding A's Question Slide, FindingAs_(\d).txt, (\w+)" )
    mFAres = mFA.search(line)
    if mFAres and state_prim == 0:
        task_type = 'FA'
        label = mFAres.group(3).lower()
        start_time = convert_to_epoch(mFAres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id 
        
    mGC = re.compile(r"(\S+), Gestalt Completion Question Slide, GestaltCompletion(\d).txt, (\w+)" )
    mGCres = mGC.search(line)
    if mGCres and state_prim == 0:
        task_type = 'GC'
        label = mGCres.group(3).lower()
        start_time = convert_to_epoch(mGCres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id         
    
    mNC = re.compile(r"(\S+), Number Comparison Question Slide, NumberComparison_(\d).txt, (\w+)" )
    mNCres = mNC.search(line)
    if mNCres and state_prim == 0:
        task_type = 'NC'
        label = mNCres.group(3).lower()
        start_time = convert_to_epoch(mNCres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id     
    
    mSX = re.compile(r"(\S+), Scattered X's Question Slide, ScatteredXs_(\d).txt, (\w+)" )
    mSXres = mSX.search(line)
    if mSXres and state_prim == 0:
        task_type = 'SX'
        label = mSXres.group(3).lower()
        start_time = convert_to_epoch(mSXres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id     

    mPT = re.compile(r"(\S+), Pursuit Test Question Slide, Pursuit_(\d).txt, (\w+)" )
    mPTres = mPT.search(line)
    if mPTres and state_prim == 0:
        task_type = 'PT'
        label = mPTres.group(3).lower()
        start_time = convert_to_epoch(mPTres.group(1))
        state_prim = 1
        curr_task_id = generate_task_ID()
        #print start_time, task_type, label, curr_task_id        
    
    mHProw = re.compile(r"'image/HiddenPatterns/(\S+)_(\w+).jpg' selected")
    mHProwres = mHProw.search(line)
    if mHProwres:
        result = mHProwres.group(2)
        if result == "true":
            num_correct += 1
        elif result == "false":
            num_incorrect += 1

    mFArow = re.compile(r" '([a-zA-Z]+)' selected")
    mFArowres = mFArow.search(line)
    if mFArowres:
        result = mFArowres.group(1).lower()
        if 'a' in result:
            num_correct += 1
        else:
            num_incorrect += 1
    
    mNCrow = re.compile(r" '([0-9]+)-([0-9]+)' selected")
    mNCrowres = mNCrow.search(line)
    if mNCrowres:
        num1 = mNCrowres.group(1)
        num2 = mNCrowres.group(2)
        if num1 != num2:
            num_correct += 1
        else:
            num_incorrect += 1
        
    mSXrow = re.compile(" \(\d+,\d+\), \(\d+,\d+\)")
    mSXrowres = mSXrow.search(line)
    if mSXrowres:
        num_correct += 1
        
    mPTrow = re.compile(", (\d+), '(\d+)'")
    mPTrowres = mPTrow.search(line)
    if mPTrowres:
        num1 = mPTrowres.group(1)
        num2 = mPTrowres.group(2)
        if PT_checker(num1, num2, label):
            num_correct += 1
        else:
            num_incorrect += 1

    mTIU = re.compile(r"TimeIsUp Slide")
    mTIUres = mTIU.search(line)
    if mTIUres:
        finished = False 
        
    mR = re.compile(r"(\S+), Rating Slide")
    mRres = mR.search(line)
    if mRres:
        end_time = convert_to_epoch(mRres.group(1))
        
    mRM = re.compile(r"(Mental|Physical|Temporal|Performance|Effort|Frustration), ([\s\w]+)")
    mRMres = mRM.search(line)
    if mRMres:
        task_load_index += TLX(mRMres.group(2).rstrip(), mRMres.group(1).rstrip())
    
    mB = re.compile(r"(Break|End) Slide")
    mBres = mB.search(line)
    if mBres and state_prim == 1:
        process_end()
        reset_task()
        
    mSCW = re.compile(r"(\d+):showing ColorWindow")
    mSCWres = mSCW.search(line)
    if mSCWres:
        if state_sec == 1:
            if curr_task_id > -1:
                print "Secondary task", curr_sec_task_id, curr_task_id, client_id, start_time_sec, "False"            
                if cursor != None:
                    cursor.execute("""INSERT INTO secondary_task (sec_task_id, prim_task_id, client_id, timestamp, opacity, finished) VALUES (%s, %s, %s, %s, %s, %s);""", (curr_sec_task_id,curr_task_id,client_id,start_time_sec,0.0,"false"))
        curr_sec_task_id = generate_sec_task_ID()
        start_time_sec = int(mSCWres.group(1))*1000
        state_sec = 1
    
    mSUC = re.compile("(\d+):user clicked on rectangle. opacity level: (\d+\.\d+)")
    mSUCres = mSUC.search(line)
    if mSUCres:
        if state_sec == 1 and curr_task_id > -1:
            opacity = mSUCres.group(2)
            state_sec = 0
            print "Secondary task", curr_sec_task_id, curr_task_id, client_id, start_time_sec, opacity, "True"
            if cursor != None:
                cursor.execute("""INSERT INTO secondary_task (sec_task_id, prim_task_id, client_id, timestamp, opacity, finished) VALUES (%s, %s, %s, %s, %s, %s);""", 
                    (curr_sec_task_id,
                    curr_task_id,
                    client_id,
                    start_time_sec,
                    opacity,
                    "true"))            


In [13]:
def generate_task_ID():
    global SEED_TASK_ID
    SEED_TASK_ID += 1
    return SEED_TASK_ID

def generate_sec_task_ID():
    global SEED_SEC_TASK_ID
    SEED_SEC_TASK_ID += 1
    return SEED_SEC_TASK_ID

# Result checker for PT task
def PT_checker(num1, num2, task_type):

    low_map = {
        "1": "3",
        "2": "4",
        "3": "1",
        "4": "2",
        "5": "6",
        "6": "5",
        "7": "9",
        "8": "10",
        "9": "7",
        "10": "8",
    }

    medium_map = {
        "1": "5",
        "2": "6",
        "3": "2",
        "4": "8",
        "5": "10",
        "6": "1",
        "7": "3",
        "8": "9",
        "9": "4",
        "10": "6",
    }    
    
    high_map = {
        "1": "9",
        "2": "8",
        "3": "10",
        "4": "7",
        "5": "6",
        "6": "5",
        "7": "4",
        "8": "3",
        "9": "1",
        "10": "2",
    }    
    
    if task_type == "low":
        return num2 == low_map.get(num1)
    elif task_type == "medium":
        return num2 == medium_map.get(num1)
    elif task_type == "hard":
        return num2 == hard_map.get(num1)
    
# TLX string value to int mapping
def TLX(value, measure_type):
    
    M_map = {
        "Zelo nizko": 1,
        "Nizko": 2,
        "Srednje": 3,
        "Visoko": 4,
        "Zelo visoko": 5,
    }
    
    P_map = {
        "Zelo dobro": 1,
        "Dobro": 2,
        "Srednje": 3,
        "Slabo": 4,
        "Zelo slabo": 5,
    }
    
    E_map = {
        "Zelo malo": 1,
        "Malo": 2,
        "Srednje": 3,
        "Veliko": 4,
        "Zelo veliko": 5,
    }
    
    if measure_type == "Physical" or measure_type == "Mental" or measure_type == "Temporal" or measure_type == "Frustration":
        return M_map.get(value)
    if measure_type == "Performance":
        return P_map.get(value)
    if measure_type == "Effort":
        return E_map.get(value)


def convert_to_epoch(value):
    datetime_object = datetime.strptime(value,'%Y:%m:%d:%H:%M:%S:%f')
    return int(datetime_object.strftime('%s'))*1000 + int(datetime_object.microsecond/1000)
    
# This function is called when a primary task end is detected
def process_end():
    global state_prim
    global start_time
    global end_time
    global task_type
    global label
    global finished
    global num_all_correct
    global num_correct
    global num_incorrect
    global curr_task_id
    global client_id
    global task_load_index
    global cursor
    
    time_on_task = end_time - start_time

    if task_type == 'HP':
        num_all_correct = 21
    if task_type == 'FA':
        #We know that there are 2 groups, 5 A-words in evey group
        num_all_correct = 10
    if task_type == 'GC':
        num_all_correct = 5
    if task_type == 'NC':
        if label == "low":
            num_all_correct = 11
        elif label == "medium":
            num_all_correct = 14
        elif label == "high":
            num_all_correct = 4
    if task_type == 'SX':
        num_all_correct = 20
    if task_type == 'PT':
        num_all_correct = 10  
    
    if curr_task_id > -1:  
        print curr_task_id, client_id, task_type, label, start_time, time_on_task, num_correct, num_incorrect, num_all_correct, task_load_index, finished
        if cursor != None:
            cursor.execute("""INSERT INTO primary_task (task_id, client_id, type, label, time_on_task, time_start, time_end, num_correct, num_incorrect, num_all_correct, task_load_index, finished) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", 
                (curr_task_id,
                client_id,
                task_type,
                label,
                time_on_task,
                start_time,
                end_time,
                num_correct,
                num_incorrect,
                num_all_correct,
                task_load_index,
                finished))


In [14]:
# This is the main cell:
# - refer it to a directory where secondary task logs are stored 
# - it iterates through each log, recognises tasks and puts them in the table
# - DON'T forget to run the following cell in order to confirm the changes to the database.

global client_id

reset_global()
reset_task()

connect_to_db()

directory = "/home/veljko/cogload/data/ijs/logs"
mSF = re.compile(r"(\w+)-secondary.txt" )

for filename in os.listdir(directory):
    if mSF.search(filename): 
        reset_user()
        reset_task()
        client_id = mSF.search(filename).group(1)
        f = open(os.path.join(directory, filename),"r")
        for line in f:
            #print line
            parse(line)
        continue
    else:
        continue

Connecting to database
  ->host=127.0.0.1 port=5432 dbname=veljkodb user=veljkopg password=pntdcn15
Connected!

Secondary task 1731 225 wjxci 1508329241000 False
Secondary task 1732 225 wjxci 1508329422000 0.125490196078 True
Secondary task 1733 225 wjxci 1508329434000 0.109803921569 True
Secondary task 1734 225 wjxci 1508329441000 0.101960784314 True
Secondary task 1735 225 wjxci 1508329449000 0.156862745098 True
Secondary task 1736 225 wjxci 1508329457000 0.156862745098 True
Secondary task 1737 225 wjxci 1508329473000 0.101960784314 True
Secondary task 1738 225 wjxci 1508329483000 0.101960784314 True
Secondary task 1739 225 wjxci 1508329490000 0.196078431373 True
Secondary task 1740 225 wjxci 1508329507000 0.101960784314 True
Secondary task 1741 225 wjxci 1508329517000 0.109803921569 True
Secondary task 1742 225 wjxci 1508329527000 0.133333333333 True
225 wjxci HP high 1508329417693 120697 21 0 21 14 True
Secondary task 1744 226 wjxci 1508329623000 0.0941176470588 True
Secondary task

Secondary task 1852 241 wjxci 1508331702000 0.188235294118 True
Secondary task 1853 241 wjxci 1508331719000 0.133333333333 True
Secondary task 1854 241 wjxci 1508331726000 0.125490196078 True
Secondary task 1855 241 wjxci 1508331738000 0.101960784314 True
241 wjxci PT low 1508331699781 44061 10 0 10 12 True
Secondary task 1856 242 wjxci 1508331796000 0.101960784314 True
Secondary task 1857 242 wjxci 1508331807000 0.0941176470588 True
Secondary task 1858 242 wjxci 1508331816000 0.141176470588 True
Secondary task 1859 242 wjxci 1508331825000 0.0941176470588 True
Secondary task 1860 242 wjxci 1508331833000 0.141176470588 True
Secondary task 1861 242 wjxci 1508331841000 0.133333333333 True
Secondary task 1862 242 wjxci 1508331854000 0.101960784314 True
Secondary task 1863 242 wjxci 1508331869000 0.141176470588 True
Secondary task 1864 242 wjxci 1508331876000 0.101960784314 True
Secondary task 1865 242 wjxci 1508331883000 0.117647058824 True
Secondary task 1866 242 wjxci 1508331892000 0.133

Secondary task 2007 258 rc1in 1508230498000 0.101960784314 True
Secondary task 2008 258 rc1in 1508230503000 0.219607843137 True
258 rc1in PT medium 1508230473737 38420 9 1 10 11 True
Secondary task 2009 259 rc1in 1508230592000 0.337254901961 True
Secondary task 2010 259 rc1in 1508230610000 0.0941176470588 True
259 rc1in PT low 1508230590845 27408 10 0 10 10 True
Secondary task 2011 260 rc1in 1508230662000 0.125490196078 True
Secondary task 2012 260 rc1in 1508230675000 0.0941176470588 True
Secondary task 2013 260 rc1in 1508230680000 0.0941176470588 True
Secondary task 2014 260 rc1in 1508230690000 0.101960784314 True
Secondary task 2015 260 rc1in 1508230695000 0.101960784314 True
Secondary task 2016 260 rc1in 1508230709000 0.141176470588 True
Secondary task 2017 260 rc1in 1508230725000 0.101960784314 True
Secondary task 2018 260 rc1in 1508230730000 0.109803921569 True
Secondary task 2019 260 rc1in 1508230737000 0.203921568627 True
260 rc1in PT high 1508230660557 86440 0 10 10 12 True
Sec

Secondary task 2124 275 ibvx8 1508430789000 0.0941176470588 True
Secondary task 2125 275 ibvx8 1508430798000 0.133333333333 True
Secondary task 2126 275 ibvx8 1508430809000 0.0941176470588 True
Secondary task 2127 275 ibvx8 1508430820000 0.0941176470588 True
Secondary task 2128 275 ibvx8 1508430833000 0.0862745098039 True
Secondary task 2129 275 ibvx8 1508430841000 0.109803921569 True
Secondary task 2130 275 ibvx8 1508430854000 0.0941176470588 True
275 ibvx8 SX high 1508430682688 182167 19 0 20 29 False
Secondary task 2132 276 ibvx8 1508430962000 0.109803921569 True
Secondary task 2133 276 ibvx8 1508430967000 0.109803921569 True
Secondary task 2134 276 ibvx8 1508430976000 0.156862745098 True
Secondary task 2135 276 ibvx8 1508430984000 0.172549019608 True
Secondary task 2136 276 ibvx8 1508431001000 0.313725490196 True
276 ibvx8 PT medium 1508430957759 65862 8 2 10 19 True
277 ibvx8 PT low 1508431088817 46849 10 0 10 23 True
Secondary task 2138 278 ibvx8 1508431191000 0.117647058824 True

Secondary task 2273 293 yljm5 1508492991000 0.101960784314 True
Secondary task 2274 293 yljm5 1508492998000 0.149019607843 True
Secondary task 2275 293 yljm5 1508493014000 0.0862745098039 True
Secondary task 2276 293 yljm5 1508493023000 0.0941176470588 True
Secondary task 2277 293 yljm5 1508493032000 0.0862745098039 True
293 yljm5 SX high 1508492953665 85190 20 0 20 10 True
Secondary task 2279 294 yljm5 1508493130000 0.235294117647 True
Secondary task 2280 294 yljm5 1508493150000 0.109803921569 True
Secondary task 2281 294 yljm5 1508493156000 0.101960784314 True
Secondary task 2282 294 yljm5 1508493167000 0.101960784314 True
Secondary task 2283 294 yljm5 1508493178000 0.0862745098039 True
Secondary task 2284 294 yljm5 1508493183000 0.101960784314 True
294 yljm5 PT medium 1508493123486 64888 9 1 10 11 True
Secondary task 2285 295 yljm5 1508493242000 0.101960784314 True
Secondary task 2286 295 yljm5 1508493248000 0.0862745098039 True
Secondary task 2287 295 yljm5 1508493261000 0.09411764

...

Secondary task 3494 411 iz3x1 1507726528000 0.0470588235294 True
Secondary task 3495 411 iz3x1 1507726538000 0.0392156862745 True
Secondary task 3496 411 iz3x1 1507726549000 0.0705882352941 True
Secondary task 3497 411 iz3x1 1507726556000 0.0862745098039 True
Secondary task 3498 411 iz3x1 1507726561000 0.0941176470588 True
Secondary task 3499 411 iz3x1 1507726575000 0.0705882352941 True
Secondary task 3500 411 iz3x1 1507726582000 0.0470588235294 True
Secondary task 3501 411 iz3x1 1507726592000 0.0470588235294 True
Secondary task 3502 411 iz3x1 1507726603000 0.0392156862745 True
Secondary task 3503 411 iz3x1 1507726608000 0.0392156862745 True
Secondary task 3504 411 iz3x1 1507726617000 0.0313725490196 True
Secondary task 3505 411 iz3x1 1507726623000 0.0705882352941 True
Secondary task 3506 411 iz3x1 1507726634000 0.0470588235294 True
Secondary task 3507 411 iz3x1 1507726647000 0.0549019607843 True
Secondary task 3508 411 iz3x1 1507726651000 0.0549019607843 True
Secondary task 3509 411 i

Secondary task 3624 426 f3j25 1508499721000 0.117647058824 True
Secondary task 3625 426 f3j25 1508499727000 0.101960784314 True
Secondary task 3626 426 f3j25 1508499733000 0.203921568627 True
Secondary task 3627 426 f3j25 1508499743000 0.149019607843 True
Secondary task 3628 426 f3j25 1508499755000 0.196078431373 True
Secondary task 3629 426 f3j25 1508499772000 0.117647058824 True
Secondary task 3630 426 f3j25 1508499785000 0.117647058824 True
Secondary task 3631 426 f3j25 1508499792000 0.101960784314 True
426 f3j25 HP low 1508499695849 112761 21 0 21 7 True
Secondary task 3633 427 f3j25 1508499859000 0.109803921569 True
Secondary task 3634 427 f3j25 1508499872000 0.0941176470588 True
Secondary task 3635 427 f3j25 1508499878000 0.0941176470588 True
Secondary task 3636 427 f3j25 1508499890000 0.0941176470588 True
Secondary task 3637 427 f3j25 1508499905000 0.101960784314 True
Secondary task 3638 427 f3j25 1508499911000 0.101960784314 True
Secondary task 3639 427 f3j25 1508499923000 0.08

Secondary task 3770 441 f3j25 1508502492000 0.078431372549 True
Secondary task 3771 441 f3j25 1508502504000 0.101960784314 True
Secondary task 3772 441 f3j25 1508502511000 0.109803921569 True
441 f3j25 PT low 1508502488165 31218 10 0 10 10 True
Secondary task 3773 442 f3j25 1508502584000 0.117647058824 True
Secondary task 3774 442 f3j25 1508502596000 0.0941176470588 True
Secondary task 3775 442 f3j25 1508502609000 0.203921568627 True
Secondary task 3776 442 f3j25 1508502623000 0.109803921569 True
Secondary task 3777 442 f3j25 1508502635000 0.0941176470588 True
Secondary task 3778 442 f3j25 1508502649000 0.0941176470588 True
Secondary task 3779 442 f3j25 1508502656000 0.125490196078 True
Secondary task 3780 442 f3j25 1508502663000 0.109803921569 True
Secondary task 3781 442 f3j25 1508502672000 0.0941176470588 True
Secondary task 3782 442 f3j25 1508502680000 0.141176470588 True
442 f3j25 PT high 1508502581861 108655 0 10 10 14 True
Secondary task 3783 443 lmdxn 1508238058000 0.1176470588

Secondary task 3915 460 lmdxn 1508240336000 0.0941176470588 True
Secondary task 3916 460 lmdxn 1508240342000 0.0941176470588 True
Secondary task 3917 460 lmdxn 1508240355000 0.101960784314 True
460 lmdxn PT high 1508240224990 135728 0 10 10 21 True
Secondary task 3918 461 6frz4 1507730321000 False
Secondary task 3919 461 6frz4 1507730530000 0.282352941176 True
Secondary task 3920 461 6frz4 1507730544000 0.196078431373 True
Secondary task 3921 461 6frz4 1507730558000 0.0862745098039 True
Secondary task 3922 461 6frz4 1507730569000 0.141176470588 True
Secondary task 3923 461 6frz4 1507730577000 0.282352941176 True
Secondary task 3924 461 6frz4 1507730594000 0.078431372549 True
Secondary task 3925 461 6frz4 1507730598000 0.274509803922 True
Secondary task 3926 461 6frz4 1507730613000 0.0627450980392 True
Secondary task 3927 461 6frz4 1507730618000 0.0705882352941 True
Secondary task 3928 461 6frz4 1507730629000 0.0705882352941 True
Secondary task 3929 461 6frz4 1507730638000 0.12549019607

Secondary task 4040 472 6frz4 1507732790000 0.133333333333 True
Secondary task 4041 472 6frz4 1507732805000 0.203921568627 True
Secondary task 4042 472 6frz4 1507732817000 0.0627450980392 True
Secondary task 4043 472 6frz4 1507732830000 0.109803921569 True
Secondary task 4044 472 6frz4 1507732842000 0.219607843137 True
Secondary task 4045 472 6frz4 1507732854000 0.0549019607843 True
472 6frz4 NC medium 1507732720265 145685 11 2 14 19 True
Secondary task 4047 473 6frz4 1507732998000 0.117647058824 True
Secondary task 4048 473 6frz4 1507733008000 0.078431372549 True
Secondary task 4049 473 6frz4 1507733013000 0.266666666667 True
Secondary task 4050 473 6frz4 1507733034000 0.0549019607843 True
473 6frz4 SX low 1507732992478 49622 20 0 20 22 True
Secondary task 4051 474 6frz4 1507733127000 0.41568627451 True
Secondary task 4052 474 6frz4 1507733146000 0.0705882352941 True
Secondary task 4053 474 6frz4 1507733159000 0.0941176470588 True
Secondary task 4054 474 6frz4 1507733168000 0.09411764

In [15]:
# Run this to commit the changes 
# Alternatively, uncomment "rollback" and comment "commit" in order to start again.

conn.commit()
#conn.rollback()
cursor.close()
conn.close()